# Описание проекта
Сервис по продаже автомобилей с пробегом разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Нужно построить модель для определения стоимости. 

**Заказчику важны**:

- качество предсказания;
- скорость предсказания;
- время обучения.

**Инструкция по выполнению проекта:**

- Загрузите и подготовьте данные.
- Обучите разные модели с различными гиперпараметрами.
- Проанализируйте скорость работы и качество моделей.

**Примечания**:

- Для оценки качества моделей применяйте метрику RMSE.
- Самостоятельно освойте библиотеку LightGBM и её средствами постройте модели градиентного бустинга.
- Время выполнения ячейки кода Jupyter Notebook можно получить специальной командой. Найдите её.
- Поскольку модель градиентного бустинга может обучаться долго, измените у неё только два-три параметра.
- Если перестанет работать Jupyter Notebook, удалите лишние переменные оператором `del`: `del features_train`

### Описание данных
**Признаки:**
- DateCrawled — дата скачивания анкеты из базы
- VehicleType — тип автомобильного кузова
- RegistrationYear — год регистрации автомобиля
- Gearbox — тип коробки передач
- Power — мощность (л. с.)
- Model — модель автомобиля
- Kilometer — пробег (км)
- RegistrationMonth — месяц регистрации автомобиля
- FuelType — тип топлива
- Brand — марка автомобиля
- NotRepaired — была машина в ремонте или нет
- DateCreated — дата создания анкеты
- NumberOfPictures — количество фотографий автомобиля
- PostalCode — почтовый индекс владельца анкеты (пользователя)
- LastSeen — дата последней активности пользователя

**Целевой признак:**
- Price — цена (евро)

# 1. Подготовка данных

In [1]:
# Импортируем все необходимые модули.
import time
import pandas as pd
import numpy as np
import lightgbm as lgb

# "Выключаем" предупреждения.
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.linear_model import Lasso, Ridge
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder, OrdinalEncoder
from sklearn.base import BaseEstimator
from sklearn.metrics import mean_squared_error as mse
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

# Создаём отдельную переменную для переменных вроде seed и random_state.
SEED = 21

In [2]:
# Загружаем данные и выводим информацию о них.
df = pd.read_csv('/datasets/autos.csv')
df.info()
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
DateCrawled          354369 non-null object
Price                354369 non-null int64
VehicleType          316879 non-null object
RegistrationYear     354369 non-null int64
Gearbox              334536 non-null object
Power                354369 non-null int64
Model                334664 non-null object
Kilometer            354369 non-null int64
RegistrationMonth    354369 non-null int64
FuelType             321474 non-null object
Brand                354369 non-null object
NotRepaired          283215 non-null object
DateCreated          354369 non-null object
NumberOfPictures     354369 non-null int64
PostalCode           354369 non-null int64
LastSeen             354369 non-null object
dtypes: int64(7), object(9)
memory usage: 43.3+ MB


,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21
5,2016-04-04 17:36:23,650,sedan,1995,manual,102,3er,150000,10,petrol,bmw,yes,2016-04-04 00:00:00,0,33775,2016-04-06 19:17:07
6,2016-04-01 20:48:51,2200,convertible,2004,manual,109,2_reihe,150000,8,petrol,peugeot,no,2016-04-01 00:00:00,0,67112,2016-04-05 18:18:39
7,2016-03-21 18:54:38,0,sedan,1980,manual,50,other,40000,7,petrol,volkswagen,no,2016-03-21 00:00:00,0,19348,2016-03-25 16:47:58
8,2016-04-04 23:42:13,14500,bus,2014,manual,125,c_max,30000,8,petrol,ford,NaN,2016-04-04 00:00:00,0,94505,2016-04-04 23:42:13
9,2016-03-17 10:53:50,999,small,1998,manual,101,golf,150000,0,NaN,volkswagen,NaN,2016-03-17 00:00:00,0,27472,2016-03-31 17:17:06


Проведём предобработку данных.

Было решено дропнуть столбцы *'DateCrawled', 'DateCreated', 'PostalCode', 'NumberOfPictures', 'LastSeen'*, в данном случае они нам не интересны и не будут использоваться в создании модели. Более того, в *'NumberOfPictures'* везде стоят нули.

In [3]:
df = df.drop(['DateCrawled', 'DateCreated', 'RegistrationMonth', 'PostalCode', 'NumberOfPictures', 'LastSeen'], axis = 1)

Посмотрим, какие данные есть в оставшихся столбцах и где пропуски стоит удалить, а где заполнить, а также удалим "мусор".

In [4]:
# Посмотрим количество пропусков в оставшихся столбцах.
df.isna().sum()

Price                   0
VehicleType         37490
RegistrationYear        0
Gearbox             19833
Power                   0
Model               19705
Kilometer               0
FuelType            32895
Brand                   0
NotRepaired         71154
dtype: int64

In [5]:
df['VehicleType'].value_counts()

sedan          91457
small          79831
wagon          65166
bus            28775
convertible    20203
coupe          16163
suv            11996
other           3288
Name: VehicleType, dtype: int64

In [6]:
df['VehicleType'] = df['VehicleType'].fillna('other')

Заменил пропуски в *'VehicleType'* на 'other', считаю оптимальным вариантом.

In [7]:
df['Gearbox'].value_counts()

manual    268251
auto       66285
Name: Gearbox, dtype: int64

Здесь заменим пропуски на 'manual', т.к. 'manual' составляет большинство.

In [8]:
df['Gearbox'] = df['Gearbox'].fillna('manual')

In [9]:
df['Model'].value_counts()

golf                  29232
other                 24421
3er                   19761
polo                  13066
corsa                 12570
                      ...  
serie_2                   8
serie_3                   4
rangerover                4
serie_1                   2
range_rover_evoque        2
Name: Model, Length: 250, dtype: int64

Здесь тоже заменим пропуски на 'other' по аналогии с *'VehicleType'*.

In [10]:
df['Model'] = df['Model'].fillna('other')

In [11]:
df['FuelType'].value_counts()

petrol      216352
gasoline     98720
lpg           5310
cng            565
hybrid         233
other          204
electric        90
Name: FuelType, dtype: int64

Пропуски в *'FuelType'* заменил на 'petrol', является самым распространённым вариантом и можно считать "опцией по умолчанию".

In [12]:
df['FuelType'] = df['FuelType'].fillna('petrol')

In [13]:
df['NotRepaired'].value_counts()

no     247161
yes     36054
Name: NotRepaired, dtype: int64

Пропуски тут заменим на 'no' по двум причинам. Первая: 'no' — большинство и вторая: этот параметр могли просто не отметить.

In [14]:
df['NotRepaired'] = df['NotRepaired'].fillna('no')

In [15]:
df['Price'].describe()

count    354369.000000
mean       4416.656776
std        4514.158514
min           0.000000
25%        1050.000000
50%        2700.000000
75%        6400.000000
max       20000.000000
Name: Price, dtype: float64

В качестве минимальной цены поставлю 99 (евро), считаю такую цену приемлимой.

In [16]:
df = df[df['Price'] >= 99]

In [17]:
df['RegistrationYear'].describe()

count    341164.000000
mean       2003.985646
std          71.315286
min        1000.000000
25%        1999.000000
50%        2003.000000
75%        2008.000000
max        9999.000000
Name: RegistrationYear, dtype: float64

Минимальным значением года является 1000, максимальным — 9999. Надо убирать. Посмотрим, какие границы мы выставим для этого столбца.

In [18]:
df['RegistrationYear'].value_counts().sort_index(ascending=False).head(40)

9999      14
9450       1
9000       2
8888       1
8500       1
8200       1
7800       1
7100       1
7000       4
6000       5
5911       2
5900       1
5600       1
5555       2
5300       1
5000      14
4800       1
4500       2
4100       1
4000       3
3800       1
3700       1
3200       1
3000       4
2900       1
2800       1
2500       2
2290       1
2222       1
2200       1
2066       1
2019      16
2018    3766
2017    9835
2016    8851
2015    1168
2014    2494
2013    4078
2012    7109
2011    9814
Name: RegistrationYear, dtype: int64

In [19]:
df['RegistrationYear'].value_counts().sort_index().head(40)

1000    22
1001     1
1039     1
1111     1
1234     4
1255     1
1300     2
1400     1
1500     3
1600     2
1602     1
1800     4
1910    36
1923     2
1925     1
1927     1
1928     2
1929     6
1930     2
1931     1
1932     3
1933     2
1934     2
1935     3
1936     3
1937    10
1938     8
1940     2
1941     2
1942     2
1943     4
1944     2
1945     4
1946     1
1947     2
1948     2
1949     1
1950    14
1951    10
1952     8
Name: RegistrationYear, dtype: int64

In [20]:
df = df[(df['RegistrationYear'] >= 1950) & (df['RegistrationYear'] <= 2020)]

In [21]:
df['Power'].value_counts()

0       34967
75      23192
60      15363
150     14106
101     12926
        ...  
1200        1
6226        1
337         1
1105        1
1090        1
Name: Power, Length: 697, dtype: int64

In [22]:
(pd.DataFrame(data=df['Power'].value_counts())).sort_index().head(35)

,Power
0,34967
1,23
2,9
3,8
4,30
5,99
6,11
7,10
8,6
9,5


In [23]:
df = df[df['Power'] <= 1000]

In [24]:
df = df[(df['Power'] >= 40) & (df['Power'] <= 1000)]

In [25]:
df['Kilometer'].describe()

count    304387.000000
mean     128649.548108
std       36460.259275
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: Kilometer, dtype: float64

Здесь всё нормально.

Для удобства, поменяем значения в *''NotRepaired* на численные.

In [26]:
df.loc[df['NotRepaired'] == 'no', 'NotRepaired'] = 0
df.loc[df['NotRepaired'] == 'yes', 'NotRepaired'] = 1

# 2. Обучение моделей

Теперь, разобьём данные на признаки и затем на несколько выборок: train, test, train_valid и valid.

In [27]:
# Выделяем целевой признак.
features = df.drop(columns='Price')
target = df['Price']

# Разбиваем данные на выборки.
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.2, random_state=SEED)
features_train_valid, features_valid, target_train_valid, target_valid = train_test_split(
    features_train, target_train, test_size=0.2, random_state=SEED)

# Проверяем размеры.
print(f'Valid shapes: {features_train_valid.shape, features_valid.shape, target_train_valid.shape, target_valid.shape}')
print(f'Train/test shapes: {features_train.shape, features_test.shape, target_train.shape, target_test.shape}')

Valid shapes: ((194807, 9), (48702, 9), (194807,), (48702,))
Train/test shapes: ((243509, 9), (60878, 9), (243509,), (60878,))


Далее, напишем функцию, которая будет возвращать время обучения, предсказания и RMSE модели. Будем проверять три модели: Ridge, CatBoostRegressor и LGBMRegressor. Для каждой модели подберём лучшие параметры через GridSearch_CV, и затем сведём все данные в одну таблицу.

In [28]:
def model_evaluation(model, features_train, target_train, features_test, target_test):
    """
    Принимает на вход модель, тренировочную и тестовую выборки,
    возвращает время обучения, время предсказания и RMSE.
    """
    
    start = time.time()    
    model.fit(features_train, target_train)
    training_time = time.time() - start
    
    start = time.time()  
    target_pred = model.predict(features_test)
    predict_time = time.time() - start

    return training_time, predict_time, mse(target_test, target_pred)**0.5

Приступим к обучению моделей.

In [29]:
# Создадим пустую переменную, куда будем записывать результаты.
results = []

### Ridge

In [30]:
# Готовим пайплайн.
pipe = Pipeline([
    ('ohe', OneHotEncoder(handle_unknown='ignore')),
    ('scaler', StandardScaler(with_mean=False)),
    ('model', Ridge(random_state=SEED))
])

params = [
    {
        'model__alpha': np.logspace(-2, 2, 20)
    } 
]

In [31]:
# Ищем лучшие параметры на кросс-валидации.
cv = KFold(n_splits=3, shuffle=True, random_state=SEED)

grid = GridSearchCV(pipe,
                    param_grid=params,
                    cv=cv,
                    scoring='neg_mean_squared_error',
                    n_jobs=-1,
                    verbose=False)

In [32]:
%%time
grid.fit(features_train_valid, target_train_valid);

CPU times: user 5min, sys: 2.92 s, total: 5min 3s
Wall time: 5min 5s


GridSearchCV(cv=KFold(n_splits=3, random_state=21, shuffle=True),
             error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('ohe',
                                        OneHotEncoder(categorical_features=None,
                                                      categories=None,
                                                      drop=None,
                                                      dtype=<class 'numpy.float64'>,
                                                      handle_unknown='ignore',
                                                      n_values=None,
                                                      sparse=True)),
                                       ('scaler',
                                        StandardScaler(copy=True,
                                                       with_mean=False,
                                                       with_std=Tru...
       6.95192796e

In [33]:
grid.best_params_

{'model__alpha': 100.0}

In [34]:
results.append(model_evaluation(grid.best_estimator_, features_train_valid, target_train_valid, features_valid, target_valid))

### LGBMRegressor

In [35]:
# Прогоняем категоральные признаки через LabelEncoder
encoder = LabelEncoder()

lgb_train = features_train_valid.copy()
lgb_test = features_valid.copy()
lgb_full_train = features_train.copy()
lgb_full_test = features_test.copy()

cat_col = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']
cat_col_num = []

for col in cat_col:
    lgb_train[col] = encoder.fit_transform(lgb_train[col])
    lgb_test[col] = encoder.transform(lgb_test[col])
    lgb_full_train[col] = encoder.fit_transform(lgb_full_train[col])
    lgb_full_test[col] = encoder.transform(lgb_full_test[col])
    cat_col_num.append(df.columns.to_list().index(col))

gbm = lgb.LGBMRegressor(boosting_type='gbdt', verbose=0, seed=SEED)


params = {
    'learning_rate': np.logspace(-3, 0, 5),
    'n_estimators': [40, 60],
    'num_leaves': [21, 31, 41],
}

grid_gbm = GridSearchCV(gbm,
                        params,
                        cv=cv,
                        scoring='neg_mean_squared_error',
                        verbose=True)

In [36]:
%%time
grid_gbm.fit(lgb_train, target_train_valid);

Fitting 3 folds for each of 30 candidates, totalling 90 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  90 out of  90 | elapsed:  7.1min finished


CPU times: user 7min 5s, sys: 2.62 s, total: 7min 8s
Wall time: 7min 13s


GridSearchCV(cv=KFold(n_splits=3, random_state=21, shuffle=True),
             error_score='raise-deprecating',
             estimator=LGBMRegressor(boosting_type='gbdt', class_weight=None,
                                     colsample_bytree=1.0,
                                     importance_type='split', learning_rate=0.1,
                                     max_depth=-1, min_child_samples=20,
                                     min_child_weight=0.001, min_split_gain=0.0,
                                     n_estimators=100, n_jobs=-1, num_leaves=31,
                                     objective=No...
                                     reg_alpha=0.0, reg_lambda=0.0, seed=21,
                                     silent=True, subsample=1.0,
                                     subsample_for_bin=200000, subsample_freq=0,
                                     verbose=0),
             iid='warn', n_jobs=None,
             param_grid={'learning_rate': array([0.001     , 0.00562341,

In [37]:
grid_gbm.best_params_

{'learning_rate': 0.1778279410038923, 'n_estimators': 60, 'num_leaves': 41}

In [38]:
results.append(model_evaluation(grid_gbm.best_estimator_, lgb_train, target_train_valid, lgb_test, target_valid))

### CatBoostRegressor

In [39]:
cbr = CatBoostRegressor(random_seed=SEED,
                        loss_function='RMSE',
                        silent=True,
                        cat_features=cat_col)

params = {
    'learning_rate': np.logspace(-3, 0, 5),
    'iterations': [40, 60],
    'depth': [6, 8, 10],
}

grid_cbr = GridSearchCV(cbr,
                        params,
                        cv=cv,
                        scoring='neg_mean_squared_error',
                        verbose=False)

In [40]:
%%time
grid_cbr.fit(features_train_valid, target_train_valid);

CPU times: user 17min 57s, sys: 2min 17s, total: 20min 14s
Wall time: 22min 2s


GridSearchCV(cv=KFold(n_splits=3, random_state=21, shuffle=True),
             error_score='raise-deprecating',
             estimator=<catboost.core.CatBoostRegressor object at 0x7fd20b10f890>,
             iid='warn', n_jobs=None,
             param_grid={'depth': [6, 8, 10], 'iterations': [40, 60],
                         'learning_rate': array([0.001     , 0.00562341, 0.03162278, 0.17782794, 1.        ])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_squared_error', verbose=False)

In [41]:
grid_cbr.best_params_

{'depth': 10, 'iterations': 60, 'learning_rate': 0.1778279410038923}

In [42]:
# Создаём таблицу с результатами.
results.append(model_evaluation(grid_cbr.best_estimator_, features_train_valid, target_train_valid, features_valid, target_valid))

# 3. Анализ моделей

Посмотрим на результаты работы различных моделей.

In [45]:
pd.DataFrame(data=results,
             index=['Ridge', 'LightGBM', 'CatBoost'],
             columns=['Время тренировки', 'Время предсказания', 'RMSE'])

,Время тренировки,Время предсказания,RMSE
Ridge,8.063422,0.109384,1990.514108
LightGBM,5.902005,0.394450,1656.142907
CatBoost,25.102294,0.140514,1663.124620


Как можно видеть, модель через LightGBM показала наименьший RSME и ей потребовалось меньше времени на обучение, чем остальным моделям.

Проверим метрику на тестовой выборке.

In [44]:
grid_gbm.fit(lgb_full_train, target_train)
mse(target_test, grid_gbm.predict(lgb_full_test))**0.5

Fitting 3 folds for each of 30 candidates, totalling 90 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  90 out of  90 | elapsed:  9.5min finished


1673.133727224133

**Вывод:** в данном случае, лучше всего использовать LGBMRegressor для построения модели.